In [ ]:
import os
gpu_num = "" # Use "" to use the CPU
os.environ["CUDA_VISIBLE_DEVICES"] = f"{gpu_num}"
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
import sionna

import tensorflow as tf
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        tf.config.experimental.set_memory_growth(gpus[0], True)
    except RuntimeError as e:
        print(e)
# Avoid warnings from TensorFlow
tf.get_logger().setLevel('ERROR')

tf.random.set_seed(1) # Set global random seed for reproducibility

import matplotlib.pyplot as plt
import numpy as np
import time

TX_POSITION = [20, 1, -2.8]
RX_POSITION = [35, 1.5, -4.5]
CENTER_X, CENTER_Y, CENTER_Z = (27, 1, -3)
SCENE_NAME = "canyon"
IMAGE_FOLDER = f"images/{SCENE_NAME}"

# Allows to exit cell execution in Jupyter
class ExitCell(Exception):
    def _render_traceback_(self):
        pass


# Import Sionna RT components
from sionna.rt import load_scene, Transmitter, Receiver, PlanarArray, Camera

# For link-level simulations
# from sionna.channel import cir_to_ofdm_channel, subcarrier_frequencies, OFDMChannel, ApplyOFDMChannel, CIRDataset
# from sionna.nr import PUSCHConfig, PUSCHTransmitter, PUSCHReceiver
# from sionna.utils import compute_ber, ebnodb2no, PlotBER
# from sionna.ofdm import KBestDetector, LinearDetector
# from sionna.mimo import StreamManagement

In [ ]:
start = time.perf_counter()
scene = load_scene(f"models/{SCENE_NAME}.xml")
scene.tx_array = PlanarArray(num_rows=8,
                          num_cols=2,
                          vertical_spacing=0.7,
                          horizontal_spacing=0.5,
                          pattern="tr38901",
                          polarization="VH")

# Configure antenna array for all receivers
scene.rx_array = PlanarArray(num_rows=1,
                          num_cols=1,
                          vertical_spacing=0.5,
                          horizontal_spacing=0.5,
                          pattern="dipole",
                          polarization="cross")

# Create transmitter
tx = Transmitter(name="tx",
              position=TX_POSITION,
              orientation=[0,0,0])
scene.add(tx)

# Create a receiver
rx = Receiver(name="rx",
           position=RX_POSITION,
           orientation=[0,0,0])
scene.add(rx)

# TX points towards RX
tx.look_at(rx)
scene_loaded = time.perf_counter()
print(f"scene load time:    {scene_loaded - start} seconds")



In [ ]:

start = time.perf_counter()

paths_simple = scene.compute_paths(
                            check_scene=False
                            )
paths_simple_time = time.perf_counter()
print(f"simple path compute time:       {paths_simple_time - start} seconds")

paths_diff = scene.compute_paths(
                            check_scene=False,
                            diffraction=True
                            )
paths_diff_time = time.perf_counter()
print(f"diffraction path compute time:  {paths_diff_time - paths_simple_time} seconds")


paths_scatter = scene.compute_paths(
                            check_scene=False,
                            scattering=True
                            )
paths_scatter_time = time.perf_counter()
print(f"scatter path compute time:      {paths_scatter_time - paths_diff_time} seconds")

paths_complete = scene.compute_paths(
                            check_scene=False,
                            diffraction=True,
                            scattering=True
                            )
paths_time = time.perf_counter()
print(f"path complete compute time:     {paths_time - paths_scatter_time} seconds")




cm = scene.coverage_map(cm_cell_size=[1.,1.], # Configure size of each cell
                        cm_center=[CENTER_X + 5, CENTER_Y, -3.0],
                        cm_orientation=[0,0,0],
                        cm_size=[35,10],
                        diffraction=True,
                        scattering=True,
                        num_samples=1e7)
cm_time = time.perf_counter()
print(f"map compute time:               {cm_time - paths_time} seconds")

In [ ]:
scene.preview(paths=paths_diff)
# scene.preview(paths=paths_scat)

In [ ]:
scene.preview(paths=paths_simple);

In [ ]:
scene.preview(paths=paths_diff);

In [ ]:
scene.preview(paths=paths_scatter);

In [ ]:
scene.preview(paths=paths_complete);

In [ ]:
scene.preview(coverage_map=cm);